In [1]:
from Transformers.Decoder import TransformerDecoder
from Transformers.Encoder import TransformerEncoder
from Transformers.PositionalEncoding import BasicPositionalEmbeddings
from Transformers.DataLoader import SampledTestatN

In [2]:
import tensorflow as tf
import string
import re

@tf.keras.utils.register_keras_serializable(name="source_standarizer")
class DefaultSourceEstandarizer(tf.keras.layers.Layer):
    def __init__(self, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
    
    def call(self, inputs, *args, **kwargs):
        strip_chars = string.punctuation
        lowercase = tf.strings.lower(inputs)
        return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

    def get_config(self):
        return super().get_config()
    
@tf.keras.utils.register_keras_serializable(name="target_standarizer")
class DefaultTargetEstandarizer(tf.keras.layers.Layer):
    def __init__(self, trainable=True, name=None, dtype=None, dynamic=False, **kwargs):
        super().__init__(trainable, name, dtype, dynamic, **kwargs)
    
    def call(self, inputs, *args, **kwargs):
        strip_chars = string.punctuation
        strip_chars = strip_chars.replace("[", "")
        strip_chars = strip_chars.replace("]", "")

        lowercase = tf.strings.lower(inputs)
        return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

    def get_config(self):
        return super().get_config()

# Model functional

In [3]:
source_input = tf.keras.Input(shape=(20, ), dtype="int64", name="source")
target_input = tf.keras.Input(shape=(20, ), dtype="int64", name="target")

source_emb = BasicPositionalEmbeddings(dim_emb=256, max_seq_length=20, max_tokens=15000)(source_input)
target_emb = BasicPositionalEmbeddings(dim_emb=256, max_seq_length=21, max_tokens=15000)(target_input)

encoded_sequence = TransformerEncoder(num_heads=1, dim_emb=256, dim_dense=512)(source_emb)

decoded_sequence = TransformerDecoder(num_heads=1, dim_emb=256, dim_dense=512)(encoded_sequence, target_emb)

decoded_sequence = tf.keras.layers.Dropout(0.5)(decoded_sequence)

next_token = tf.keras.layers.Dense(units=15000, activation="softmax")(decoded_sequence)

model = tf.keras.Model(inputs=[source_input, target_input], outputs=next_token)

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2023-05-27 15:24:43.703423: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-27 15:24:43.703548: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-3), metrics=["accuracy"])

In [5]:
data_loader = SampledTestatN(path="./eng-ita/eng-ita.txt",
                              max_tokens_source=15000,
                              max_tokens_target=15000,
                                seq_length_source=20,
                                  seq_length_target=20,
                                    source_preprocess=None,
                                      target_preprocess=None
                            )

In [6]:
DefaultSourceEstandarizer()([["asdasd asdas"], ["asdasda aaad aa"]])

<tf.Tensor: shape=(2, 1), dtype=string, numpy=
array([[b'asdasd asdas'],
       [b'asdasda aaad aa']], dtype=object)>

In [7]:
train_dataset, val_datasets, test_dataset = data_loader(batch_size=32)

Reading file... (1/4)
Splitting data... (2/4)
Total records: 1000
Building vocabulary... (3/4


2023-05-27 15:24:59.060585: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-05-27 15:24:59.098314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-27 15:24:59.170974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Building tensorflow datasets... (4/4)
Done!


In [8]:
stop_early_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

save_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="test_function_model_best", save_best_only=True, save_weights_only=False)

model.fit(train_dataset, validation_data=val_datasets, epochs=5, callbacks=[stop_early_callback, save_checkpoint])

Epoch 1/5


2023-05-27 15:25:04.740030: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 4.0522 - accuracy: 0.8176

2023-05-27 15:25:07.869424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: test_function_model_best/assets


...

24/24 [==============================] - ETA: 0s - loss: 0.4724 - accuracy: 0.9020

INFO:tensorflow:Assets written to: test_function_model_best/assets


INFO:tensorflow:Assets written to: test_function_model_best/assets


24/24 [==============================] - 4s 190ms/step - loss: 0.4724 - accuracy: 0.9020 - val_loss: 0.7792 - val_accuracy: 0.8997


/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 source (InputLayer)            [(None, 20)]         0           []                               
                                                                                                  
 basic_positional_embeddings (B  (None, 20, 256)     3845120     ['source[0][0]']                 
 asicPositionalEmbeddings)                                                                        
                                                                                                  
 target (InputLayer)            [(None, 20)]         0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, 20, 256)     658432      ['basic_positional_embeddings

In [10]:
model.save("stupidmodel")

INFO:tensorflow:Assets written to: stupidmodel/assets


INFO:tensorflow:Assets written to: stupidmodel/assets
/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [11]:
tf.keras.models.load_model("stupidmodel")

In [12]:
x_source = tf.keras.Input(shape=(), dtype=tf.string, name="source")
x_target = tf.keras.Input(shape=(), dtype=tf.string, name="target")

source_vect = data_loader.source_tokenizer(x_source)
target_vect = data_loader.source_tokenizer(x_target)

next_token = model({"source":source_vect, "target":target_vect})

new_model = tf.keras.Model(inputs=[x_source, x_target], outputs=next_token)

In [16]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 source (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 target (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 source_vectorizer (TextVectori  (None, 20)          0           ['source[0][0]',                 
 zation)                                                          'target[0][0]']                 
                                                                                                  
 model (Functional)             (None, 20, 15000)    13257368    ['source_vectorizer[0][0]',

In [17]:
new_model.save("stupidmodewithvectorizer")

INFO:tensorflow:Assets written to: stupidmodewithvectorizer/assets


INFO:tensorflow:Assets written to: stupidmodewithvectorizer/assets
/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/engine/functional.py:1384: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/Users/matiasgonzalez/miniforge3/envs/tf_2_9_metal_0_5_keras/lib/python3.10/site-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [18]:
loaded_new_model = tf.keras.models.load_model("stupidmodewithvectorizer")

2023-05-27 15:33:02.953126: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-27 15:33:02.958099: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [19]:
loaded_new_model({"source":tf.constant(["hello"]), "target":tf.constant(["[start]"])})

2023-05-27 15:33:07.486857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-27 15:33:07.514573: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


<tf.Tensor: shape=(1, 20, 15000), dtype=float32, numpy=
array([[[8.7574112e-01, 1.8016009e-07, 1.7463861e-07, ...,
         2.8002722e-07, 2.3988505e-07, 2.7208344e-07],
        [8.8002473e-01, 1.7595707e-07, 1.7028388e-07, ...,
         2.7292870e-07, 2.3406155e-07, 2.6583197e-07],
        [8.8442200e-01, 1.7187486e-07, 1.6633965e-07, ...,
         2.6634896e-07, 2.2858836e-07, 2.5999026e-07],
        ...,
        [8.9173722e-01, 1.6582938e-07, 1.5985616e-07, ...,
         2.5574511e-07, 2.2038448e-07, 2.5106007e-07],
        [8.9177632e-01, 1.6579584e-07, 1.5983268e-07, ...,
         2.5570557e-07, 2.2036157e-07, 2.5100240e-07],
        [8.9191872e-01, 1.6569363e-07, 1.5972486e-07, ...,
         2.5551455e-07, 2.2019947e-07, 2.5086152e-07]]], dtype=float32)>